<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


###### Мы сделали общий обзор данных. В представленном датасете 5000 строк.  

Разделим обучающие данные и целевые признаки.

In [6]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [7]:
display(features.head(5))
display(target.head(5))

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


0    0
1    1
2    0
3    0
4    0
Name: Страховые выплаты, dtype: int64

In [8]:
features.shape

(5000, 4)

## Умножение матриц

Вопрос: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** не изменится

**Обоснование:** 

Чтобы доказать, что качество линейной регрессии не изменится, мы должны убедиться в следующем:

выражение
$$
a_1 = XPw = XP ((XP)^T (XP))^{-1} (XP)^T y
$$
равно следующему выражению
$$
a = Xw = X (X^T X)^{-1} X^T y 
$$

Воспользуемся следующим свойством матриц $(AB)^T = B^{T} A^{T}$

$$
XP ((XP)^T (XP))^{-1} (XP)^T y = XP (P^T X^T XP)^{-1} P^T X^T y = XP (P^T (X^T XP))^{-1} P^T X^T y
$$

воспользуемся следующим свойством матриц $(AB)^{-1} = B^{-1} A^{-1}$, учитывая при этом, что обратной матрицы $ X^{-1} $ не существует, так как матрица признаков не квадратная

$$
XP (P^T (X^T XP))^{-1} P^T X^T y = XP (X^T XP)^{-1} (P^T)^{-1} P^T X^T y = XP ((X^T X)P)^{-1} (P^T)^{-1} P^T X^T y = 
 XP P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

заменим $P P^{-1}$ на $E$ за счет следующего свойства матрицы $AA^{-1} = A^{-1}A = E $

$$
XP P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = X E (X^T X)^{-1} E X^T y
$$

Как известно $AE = EA = A$, тогда

$$
X E (X^T X)^{-1} E X^T y = X (X^T X)^{-1} X^T y
$$

Что и требовалось подтвердить

Таким образом, мы доказали тождество двух выражений, что подтверждает, что умножение матрицы признаков на любую обратимую матрицу не меняет предсказаний, а значит качество линейной регрессии

## Алгоритм преобразования

**Алгоритм**

Мы умножаем матрицу признаков на произвольную обратимую матрицу. Таким образом, мы получаем новую матрицу признаков с измененными конфиденциальными данными. При этом качество предсказаний не изменится.

**Обоснование**

Доказано выше

## Проверка алгоритма

Определим качество модели при неизмененных признаках

In [9]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
r2_score_origin = r2_score(target, predictions)
print(r2_score_origin)

0.42494550286668


Теперь для проверки алгоритма создадим произвольную матрицу $Z$

In [10]:
np.random.seed(42)
random_matrix = np.random.randint(1, 20, size=(4, 4))
print(random_matrix)

[[ 7 15 11  8]
 [ 7 19 11 11]
 [ 4  8  3  2]
 [12  6  2  1]]


Убедимся, что она обратимая

In [11]:
while np.linalg.det(random_matrix) == 0:
    random_matrix = np.random.randint(1, 20, size=(4, 4))

Умножим матрицу признаков на нашу матрицу

In [12]:
new_features = features @ random_matrix

In [13]:
new_features.head()

,0,1,2,3
0,198706.0,397600.0,149264.0,99660.0
1,152334.0,304880.0,114508.0,76507.0
2,84203.0,168551.0,63319.0,42319.0
3,166971.0,334011.0,125335.0,83633.0
4,104603.0,209347.0,78619.0,52516.0


Вышепредставленная матрица говорит нам примерно ничего. Конфиденциальные данные в безопасности.

In [14]:
model = LinearRegression()
model.fit(new_features, target)
predictions = model.predict(new_features)
r2_score_changed = r2_score(target, predictions)
print(r2_score_changed)

0.424945502866683


Сравним результаты:

In [15]:
comparison = pd.DataFrame(data=[r2_score_origin, r2_score_changed],
                          columns=['values'],
                          index=['r2_score_origin', 'r2_score_changed'])
display(comparison)

,values
r2_score_origin,0.424946
r2_score_changed,0.424946


Как видно выше, качество модели не изменилось